In [20]:
import numpy as np
import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, DistilBertForSequenceClassification
from iDistilbert import iDistilBertForSequenceClassification
import torch.quantization
from torch.utils.data import DataLoader
from tqdm import tqdm

In [36]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer

student_id = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(student_id)

dataset = load_dataset("imdb")

def pre_process(examples):
    tokenized = tokenizer(examples["text"], truncation=True, padding = 'max_length',  max_length=512)
    tokenized['label'] = examples['label']
    return tokenized

tokenized_data = dataset.map(pre_process, batched = True)
test_dataset = tokenized_data["test"].with_format("torch")
test_dataloader = DataLoader(test_dataset, batch_size=8)

In [40]:
model = iDistilBertForSequenceClassification.from_pretrained('./models', load_in_8bit=True)
model.eval()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


ValueError: `.float()` is not supported for quantized model. Please use the model as it is, since the model has already been casted to the correct `dtype`.

In [39]:
total_correct = 0
total_samples = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")        
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
                
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
                
        total_correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)
        
accuracy = total_correct / total_samples
return accuracy

Evaluating:   1%|▏         | 42/3125 [01:10<1:26:30,  1.68s/it]


KeyboardInterrupt: 